## $\quad$ $\quad$  $\quad$  $\quad$  $\quad$  $\quad$  Gasset Mathieu - Guerin Cyril - Pot Eline

-----------------------------------------------------------------------------------------------------------------------------------------------------------------

## $\quad$ Adaptive Tuning Of Hamiltonian Monte Carlo Within Sequential Monte Carlo

### Sorbonne Université $\quad$ $\quad$ $\quad$ $\quad$ $\quad$ $\quad$  $\quad$  $\quad$  $\quad$ $\quad$  $\quad$  $\quad$  $\quad$  $\quad$  $\quad$  $\quad$ $\quad$  Modèles génératifs

### Introduction

Les SMC offrent une stratégie pour approcher une distribution cible en échantillonnant des particules depuis une distribution initiale et en les déplaçant à travers une séquence de distributions intermédiaires. Cette approche permet l'estimation de constantes de normalisation, ce qui est utile pour la sélection de modèles. Cependant l'efficacité des échantillonneurs SMC dépend grandement des noyaux de Markov utilisés. Ainsi cet article explore l'intégration des noyaux sélectionné par la méthode de Monte Carlo Hamiltonienne et donc de ces avantages par rapport à des méthodes plus classiques comme MCMC.

### $\textbf{Contexte}$

Soit $(\Omega, \mathcal{F}, \mathbb{P})$ un espace de probabilité. Soit $X$ : $(\Omega,\mathcal{F}) \rightarrow \mathbb{R}^d$ de loi a densite $p$ pour la mesure de Lebesgue et $Y$ : $(\Omega,\mathcal{F}) \rightarrow E$ avec (E un espace Polonais) que l'on observe, de loi sachant $X$ a densite $\pi(. | X)$ par rapport a une mesure $\mu$ de $E$. On introduit $\forall i \in \llbracket 1,N \rrbracket, Y_i$ des copies i.i.d de $Y$. On définit la vraisemblance de $Y$ comme $l(Y_1, Y_2, \ldots, Y_N|X) = \prod_{i=1}^{N} \pi{(Y_i|X)}$.
La formule de Bayes nous dit que la loi $\Pi(.|Y)$ de $X \, | \, Y$ à une densité par rapport à la mesure de lebesgue, qui vaut

$$
\pi(x|Y) = \frac{\pi(Y|x)p(x)}{\int_{\mathbb{R}^d} p(u)p(Y|u)d\lambda_d(u)}
$$

Ainsi nous avons les nommages suivantes : $\newline$

- $\pi(.|Y)$ est la distribution a posteriori $X \, | \, Y$,
- $\pi(.|X)$ est la distribution de $Y \, | \, X$,
- $p(.)$ est la distribution a priori de $X$,
- Le dénominateur est la constante de renormalisation.

Soit une fonction test $\varphi : \mathbb{R}^d \rightarrow \mathbb{R}$ tel que $\varphi \in L^1(\mathbb{R})$ par rapport à $\Pi(.|Y)$. On souhaite approximer la valeur :
$$
\int_{\mathbb{R}^d} \varphi(x) \pi(x|Y) d\lambda_d(x).
$$

On souhaite également estimer la vraisemblance marginale $Z = \int_{\mathbb{R}^d} \pi(y|x) p(x) dx$

### $\textbf{I - Sequential Monte-Carlo avec la méthode MCMC}$

Les méthodes de Monte-Carlo séquentiel consistent en l'introduction d'une séquence de distributions intermédiaires, $\forall$ t $\in  \llbracket 0,T \rrbracket$, $ \pi_0, \ldots, \pi_t, \ldots, \pi_T $, et telles que $\pi_0$ soit la loi a priori $\pi$ et soit $\textbf{facile à échantillonner}$, nous avons aussi $ \pi_T = \pi(.|Y) $

On considère une suite $\forall t \in \llbracket 0,T \rrbracket$, $\lambda_{t}$ avec $0 = \lambda_0 < \ldots < \lambda_t < \ldots < \lambda_T = 1$ qui va nous permette d'introduire des distributions dites "tempérées". En effet, par proportionnalité, puisque le dénominateur est une constante, nous avons : $\pi_t(.|X) \propto p(.)\pi(Y|.)$, en rajoutant l'exposant sur la vraisemblance nous obtenons les distributions tempérées : $\pi_t(.|X) \propto p(.)\pi(Y|.)^{\lambda_t}$.

L'utilisation de ces types de distributions est utile pour la raison suivante : 

* $\textbf{Transition douce vers la distribution a posteriori :}$ L'augmentation graduelle de $ \lambda_t $ qui prends ces valeurs entre 0 et 1, conduit à une série de distributions intermédiaires qui se déplacent progressivement de la distribution a priori vers la distribution a posteriori. Quand $\lambda_t$ augmente quand t augmente, la vraisemblance commence à influencer de plus en plus la forme de la distribution intermédiaire. La transition est progressive et permets donc d'éviter de rester coincé dans des extremas locaux.

<!-- $\textbf{3 - Sequential Monte Carlo}$ : -->

Pour construire un échantillonneur à partir de cette méthode, on utilise la méthode SMC (Sequential Monte Carlo). 
<!-- Cette méthode consite à estimer progressivement notre distribution cible $\pi_T$ en utilisant une suite de distributions intermédiaires $\pi_t$ avec $t \in \llbracket 1, T \rrbracket$ et d'ensemble de particules. -->

Dans la suite, on introduit à chaque temps $t \in \llbracket 1, T \rrbracket$ une distribution tempérée et non normalisée associée à $\pi(x)$, qui s'écrit : $\gamma_t(x) = p(x) l(y|x)^{\lambda_t}$. 

Son facteur de normalisation vaut $Z_t = \int_{\mathbb{R}^d} \gamma_t(x) dx$.


### Présentation de l'algorithme SMC
L'algorithme SMC se déroule de la façon suivante:

**Initialisation :** On commence, au temps $t=1$, avec un ensemble de $N$ particules initiales $(x_1^{i})_{i=1}^N \sim \pi$ qui représentent la distribution a priori de l'état du système qui sont i.i.d. On dispose également de la première température $\lambda_0 = 0$. On a donc $\gamma_{0}(.) = p(.)$

**A chaque itération** $t \in \llbracket 2, T \rrbracket $ **:**

On dispose de particules $(x_t^{i})_{i=1}^N$ et qu'on suppose distribuées suivant $\pi_{t-1}$

On choisit la prochaine température $\lambda_t \in (\lambda_{t-1}, 1]$.
On pose $\gamma_{t}(.) = p(.)\pi(Y|.)^{\lambda_t}$. 

On calcule les poids des particules définis par : $w_t^{i}=\frac{\gamma_t(x_t^{i})}{\gamma_{t-1}(x_t^{i})}$ pour chaque particule $i$.
Ces poids nous indiquent si chaque nouvelle particule est pertinente et représentative dans l'approximation de la prochaine distribution $\pi_t$, partant de la particule précédente. Notons que leur moyenne permet d'estimer le rapport $\frac{Z_t}{Z_{t-1}}$ par Monte Carlo.

On ré-échantillonne les particules en fonction de leur poids pour obtenir un nouvel ensemble $\{\tilde{x}_{t}^{i}\}$. Cela permet d'enlever les particules les moins informatives, et de conserver les particules les plus probablement correctes. Cela permettra d'avoir une meilleure approximation de $\pi_t$.

<!-- Supposons qu'au temps $t - 1$ on ait une approximation de particules de poids égaux $\{\tilde{x}_{t-1}^{i}\}_{i=1:N}$ de $\pi_{t-1}$ est disponible, avec des duplicatas possibles parmi les particules.  -->


Ce nuage de particules est ensuite déplacé avec un noyau de Markov $K_{t+1}^h$, qui laisse la distribution $\pi_t$ invariante : pour chaque $i$,
$$
x_{t+1}^{i} \sim K_{t+1}^h(\tilde{x}_{t}^{i}, dx).
$$

Notons que par construction du noyau par une méthode MCMC, notre noyau vérifie les conditions de la proposition suivante :
**Proposition :** On considère un espace d'états E quelquonque. Soit $(X_{n})_{n \geq 0}$, une chaine de markov de noyau markovien P, irréductible, qui vérifie la condition de Doeblin, alors pour toute loi initiale $v$ de $X_0$, la loi de $X_n$ converge en variation vers une probabilité $\pi$. On a aussi que $\pi$ est l'unique mesure de probabilité invariante pour $(X_{n})_{n \geq 0}$

Ainsi, les particules parcourent l'espace d'état tout en se rapprochant de la loi cible $\pi_{t}$
Par conséquent, un ensemble de nouvelles particules $\{x_{t+1}^{i}\}_{i=1:N}$ est obtenue, de distribution proche de $\pi_t$. 

**Sortie :** A la fin, on retourne l'ensemble des échantillons et de leur poids calculés à toutes les itérations : $\{x_t^{i}, w_t^{i}\}$.
On retourne aussi chaque estimateur des ratio des constantes de normalisations $\widehat{\frac{Z_t}{Z_{t-1}}} = \frac{1}{N} \sum_{i=1}^N w_t^{i}$ qui est un estimateur de Monte Carlo du ratio entier en échantillonnant sous $\pi_{t-1}$.
On note qu'on a donc accès à un estimateur du ratio $\frac{Z_T}{Z_0}$

Ainsi en faisant $\textbf{T+1}$ itérations, on obtient des échantillons qui suivent la loi a posteriori $\pi_T = \pi(.|Y)$.
Il ne nous reste donc plus qu'à faire une moyenne pour obtenir l'esperance de $\varphi$ sous $\pi_T$ par la loi forte des grands nombres.

## $\textit{Remarque}$:

L'algorithme, d'après l'article, présente et propose une autre façon d'obtenir la loi a posteriori via un importance sampling. En effet, une fois l'algorithme terminé en $\textbf{T}$ itérations, on possède des échantillons de la loi à $\textbf{T-1}$. On a donc accès à des échantillons avec des poids ${\{x_t^{i}, w_t^{i}\}}_{i=1,...N ; t=1,...,T}$ qui vont servir à estimer la valeur de $\varphi$ en espérance selon les lois cibles $\pi_t$. 

Pour ce faire, on utilise les notions d'Importance Sampling :

Il faut d'abord s'assurer d'avoir l'hypothèse de domination entre $\pi_{t+1}$ et $\pi_{t}$. C'est à dire : $\forall x \in \mathbb{R},  \pi_{t-1}(x) = 0$ alors $\pi_{t}(x) = 0$. Ici, c'est bien le cas puisque $\pi_t$ est construit à partir de $\pi_{t-1}$. 
<!-- puisque le temps t n'existe que si le temps t-1 a existé. -->
Ainsi en considérant la sortie de l'algorithme et par définition de $\omega_{t}^{i}$, on a, par la loi forte des grands nombres: 
$$
\sum_{i=1}^N \frac{\varphi(x_{t}^{i})\omega_{t}^{i}}{\sum_{l=1}^N \omega_{t}^{l}} \overset{p.s.}{\longrightarrow} \mathbb{E}_{\pi_{t}}[\varphi(x)].
$$


Un poids élevé signifie que l'échantillon $x_{t}^{i}$ est plus représentatif de la distribution cible.

Cette façon de procéder fonctionnerait tout aussi bien que la méthode qui consiste à faire $\textbf{T+1}$ itérations. Dans la suite, nous avons choisi d'implémenter l'algorithme avec $\textbf{T+1}$ itérations.

$\textbf{Tuning des lambda}$


Les $\lambda$ sont adaptatifs, ainsi une méthode commune basé sur $\textit{Jasra et al., 2011; Schafer and Chopin,
2013}$ et $\textit{effective sample size - Kong et al., 1994}$, $\textit{Agapiou et al., 2017}$, est de regarder la mesure de performance pour les estimateurs d'importance sampling, noté $\textit{ESS}$ :
$$
ESS(\lambda_t) = \frac{\left(\sum_{i=1}^N w_t^i\right)^2}{\sum_{i=1}^N (w_t^i)^2},
$$
où $ w_t^i = \frac{\gamma_t(x_t^i)}{\gamma_{t-1}(x_t^i)}$ $\newline$ L'ESS est une approximation de Monte Carlo de $ N/(1 + \chi^2(\pi_t, \pi_{t-1})) $ où $\chi^2(\pi_t, \pi_{t-1})$ est la divergence $\chi^2$ de $\pi_{t-1}$ à $\pi_t$.
Ainsi $\lambda_t$ est choisit en résolvant (en $\lambda$) l'équation $ ESS(\lambda) = \alpha N $, pour une valeur $\alpha \in (0,1)$ choisie.


## $\textbf{Expérimentation - SMC}$


Nous avons réimplémenté les algorithmes décrits par l'article (avec parfois quelques simplifications) et on les a testés sur des données générées avec $X\sim\mathcal{U}([0,1])$ et $Y\sim \mathcal{B}(X)^{\otimes n}$.

In [1]:
import torch
import numpy as np
rng = np.random.default_rng()
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def SMC(Y, sample_X, N, p, l, K_markov, h, should_continue_markov, lambda_get):
    X = sample_X(N)
    t, lambda_tm1, lambda_t = 1, 0, lambda_get(1)

    omegas = []
    while lambda_t <= 1:
        omega = l(Y, X)**(lambda_t - lambda_tm1)
        omegas.append(omega)
        omega = omega.squeeze().cpu().numpy()
        Xtilde = torch.from_numpy(
            rng.choice(X.cpu(), size=N, p=omega/omega.sum())
        ).to(device)

        X, Xk = Xtilde, X.clone()
        while should_continue_markov(Xk):
            X = K_markov(X, lambda x : p(x) * l(Y, x), h)
            Xk = torch.cat((Xk, X), 1)

        lambda_tm1 = lambda_t
        lambda_t = lambda_get(t)
        h.next_t()
        t = t + 1
    return X, omegas

cpu
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import scipy.stats

def sample_X(N):
    return torch.rand((N, 1), device=device)

def p(X):
    return (X >= 0) * (X <= 1)

def sample_Y(n):
    X = torch.rand(1, device=device)
    Y = (torch.rand(n, device=device) <= X).float()
    return X, Y

def l(Y, X):
    X = X.repeat(1, Y.shape[0])
    return (X**Y * (1 - X)**(1 - Y)).prod(axis=-1, keepdim=True)

def K_markov(X, pi, h):
    # Genere une les etats suivants des X pour un noyeau markovien de pi
    N = X.shape
    XS = (torch.rand(N, device=device) * (h.sup_bound - h.inf_bound)
             + h.inf_bound + X)
    U = torch.rand(N, device=device)
    return torch.where(U <= pi(XS)/pi(X), XS, X)

def should_continue_markov(Xk):
    return Xk.shape[1] <= 20

def lambda_get(N, t):
    return t/N

class ClassicMcmcParameters:
    def __init__(self, inf_bound, sup_bound):
        self.inf_bound = inf_bound
        self.sup_bound = sup_bound

    def next_t(self):
        pass


N, n = 1000, 5
XR, Y = sample_Y(n)
print(f"Vrai X : {XR}, Y : {Y}")
print(f"Esperance a posteriori réelle : {scipy.stats.beta(1 + Y.sum().cpu(), 1 + n - Y.sum().cpu()).mean()}")

X, omegas = SMC(
    Y, sample_X, N, p, l,
    K_markov, ClassicMcmcParameters(-0.1, 0.1),
    should_continue_markov, lambda t : lambda_get(10, t)
)
print(f"Estimateur de l'ésperance a posteriori selon SMC : {X.mean()}")

Vrai X : tensor([0.3528]), Y : tensor([0., 0., 0., 0., 1.])
Esperance a posteriori réelle : 0.2857142984867096
Estimateur de l'ésperance a posteriori selon SMC : 0.27261778712272644


### $\textbf{II - Hamiltonien MonteCarlo}$

La méthode de Hamiltonian MonteCarlo (HMC) est une méthode d'échantillonnage utilisée pour simuler des échantillons à partir de distributions de probabilité complexes et incalculable, en particulier dans des espaces de haute dimension. $\newline$
La méthode consiste à construire un noyau markovien de Metropolis Hastings avec pour probabilité invariante :
$$
\Pi = \pi \otimes \mathcal{N}(0, M)
$$
avec M une matrice symétrique définie positive.

La méthode HMC construit alors le noyau markovien $P$, qui est défini comme : 
$$
P \bigl( (p_t, q_t), (dp_{t+1}, dq_{t+1}) \bigr) = \frac{1}{(2 \pi)^{\frac{d}{2}}|M|^{\frac{1}{2}}} e^{-\frac{1}{2} q^{\top}_{t+1} M q_{t+1}} \delta_{\phi(p_t, q_{t+1})}(p_{t+1})
$$


Avec $\phi = s \circ \varphi_{\kappa} : \mathbb{R^d} \times \mathbb{R^d} \rightarrow \mathbb{R^d} \times \mathbb{R}^d$. Et où $\varphi_{\kappa}$ est le résultat au bout d'un temps $\kappa$ d'une particule de position initiale $p_t$, d'énergie cinétique $q_t$, avec une énergie potentielle suivant la fonction $-log(\pi)$  et  $\begin{array}{l c c c} s : & \mathbb{R^d} \times \mathbb{R^d} & \longrightarrow & \mathbb{R^d}  \times \mathbb{R^d} \\ & (p,q) & \longrightarrow & (p,-q) \end{array}$ 

Cette définition de $P$ revient à dire que pour un état $(p_{t},q_{t})$, le couple de variables aléatoires $(p_{t+1},q_{t+1})$ résultant du noyau markovien pour cet état vérifie :
$$
q_{t+1} \sim \mathcal{N}(0, M),\qquad p_{t+1} = \phi(p_{t},q_{t+1})
$$


Dans cette situation, on obtient que la fonction $\phi$ est une involution qui préserve le volume et conserve la quantité hamiltonienne $H$ définie par :
$$
H(p,q) = -log(\pi) + \frac{1}{2}q^{T}M^{-1}q
$$ 

Dans ce cas idéal, on a donc un noyau markovien $P$ de Métropolis Hastings avec pour probabilité invariante $\Pi$ .

Malheureusement en pratique, il est impossible de calculer $\varphi_{\kappa}$. On utilise donc une approximation, appelée Leapfrog integrator, qui est la suivante :
Prenons $L \in \mathbb{N^*}$, $\epsilon > 0$ tel que $\kappa$ = $L \epsilon$, et posons $\varphi_{\epsilon,L}(p_t, q_t) = (p_{t+\kappa}, q_{t+\kappa})$. Les $(p_{t+\kappa}, q_{t+\kappa})$ sont calculés de manière récursive, en prenant $\tau = t + k \epsilon$ avec $k = 0, ..., L-1$,  :
$$
q_{\tau+\epsilon/2} = q_{\tau} + \frac{\epsilon}{2} \nabla_p (log(\pi(p_\tau))), \newline


p_{\tau+\epsilon} = p_{\tau} + \epsilon M^{-1} q_{t+\epsilon/2}, \newline


q_{\tau+\epsilon} = q_{\tau+\epsilon/2} + \frac{\epsilon}{2} \nabla_p (log(\pi(p_{\tau+\epsilon}))),


$$
La méthode préserve bien le volume et est une involution mais elle ne préserve pas la quantité hamiltonienne. Ainsi le noyau découlant de Métropolis Hastings dans ce cas a pour probabilité d'acceptation : 
$$ 
\min \left(1, \frac{\Pi(p_{t+\kappa}, q_{t+\kappa})}{\Pi(p_{t}, q_{t})}\right) = \min \left(1, \frac{e^{-H(p_{t+\kappa}, q_{t+\kappa})}}{e^{-H(p_{t}, q_{t})}}\right) = \min \biggl( 1, e^{-H(p_{t+\kappa},q_{t+\kappa}) + H(p_{t},q_{t})} \biggr) = \min \biggl( 1, e^{\Delta E_\kappa} \biggr)
$$ 
 

## $\textit{Remarque:}$
L'article stipule que $\Delta E_k = H(\hat{q}_{\tau+k}, \hat{p}_{\tau+k}) - H(q_\tau, p_\tau)$ or si on prend le cas $k$ = 1, et si $\pi$ vaut 0 au temps suivant, alors on a $H$ = +$\infty$, c'est-à-dire qu'on accepte tout le temps là où on devrait refuser. Ainsi, il semble plus raisonnable de prendre l'opposé : $\Delta E_k =: H(q_\tau, p_\tau) - H(\hat{q}_{\tau+k}, \hat{p}_{\tau+k})$. 

Ici, pour reprendre les notations de l'article, nos particules ont une position notée $x_t^i$, et on une vitesse qu'on notera $p_t^i$  pour $i=1,...,N$ et $t \in \llbracket 1,T\rrbracket$

## $\textbf{Expérimentation HMC}$

In [3]:
def K_HMC(X, pi, h):
    p0, q0 = X.clone().requires_grad_(), torch.randn(X.shape, device=device)
    p, q = p0, q0

    for k in range(h.L):
        grad_p = torch.where(pi(p) > 0, torch.autograd.grad(
            torch.log(pi(p)), p, grad_outputs=torch.ones_like(p), create_graph=True
        )[0], torch.zeros(p.shape, device=device))

        q = q + h.epsilons/2 * grad_p
        p = p + h.epsilons * q

        grad_p = torch.where(pi(p) > 0, torch.autograd.grad(
            torch.log(pi(p)), p, grad_outputs=torch.ones_like(p), create_graph=True
        )[0], torch.zeros(p.shape, device=device))

        q = q + h.epsilons/2 * grad_p

    p0, q0 = p0.detach(), q0.detach()
    p, q = p.detach(), q.detach()
    
    DeltaE = ((-torch.log(pi(p0)) + 1/2 * torch.einsum('ij,jk,ik->i', q0, h.M, q0).unsqueeze(1))
              - (-torch.log(pi(p)) + 1/2 * torch.einsum('ij,jk,ik->i', q, h.M, q).unsqueeze(1)))
    h.update_parameters(p0, q0, p, q, DeltaE)

    U = torch.rand(p.shape, device=device)
    return torch.where(torch.log(U) <= DeltaE, p, p0)

In [4]:
N, n = 500, 5
XR, Y = sample_Y(n)
print(f"Vrai X : {XR}, Y : {Y}")
print(f"Esperance a posteriori réelle : {scipy.stats.beta(1 + Y.sum().cpu(), 1 + n - Y.sum().cpu()).mean()}")

class ClassicHmcParameters:
    def __init__(self, epsilon, L, M, N):
        self.epsilons = torch.ones((N, 1), device=device) * epsilon
        self.M = M
        self.L = L

    def update_parameters(self, p0, q0, p, q, DeltaE):
        pass

    def next_t(self):
        pass

X = sample_X(N)
h = ClassicHmcParameters(0.01, 10, torch.tensor([[1.]], device=device), N)
for i in range(N):
    X = K_HMC(X, lambda x : p(x) * l(Y, x), h)
print(f"Estimateur de l'esperance a posteriori selon HMC : {X.mean()}")

Vrai X : tensor([0.9618]), Y : tensor([1., 1., 1., 1., 1.])
Esperance a posteriori réelle : 0.8571428656578064
Estimateur de l'esperance a posteriori selon HMC : 0.8547757863998413


### $\textbf{III - Calibration par HMC de SMC}$

**Calibration de M:**
Dans cette partie, on calibre la matrice de covariance M utilisé dans la méthode HMC à chaque itération $t$ :
$$
M_t = \text{diag}(\widehat{\text{Var}}_{\pi_t}[p_t])^{-1}
$$
Avec $\widehat{\text{Var}}_{\pi_t}[p_t]$ l'estimation de la matrice de covariance sous la loi $\pi_{t}$.

Le fait de calibrer cette matrice à partir de particules dont on dispose au temps $t$ permet d'exploiter l'information générée par ces particules. Par ailleurs, le fait de prendre une matrice diagonale permet une application plus aisée en grande dimension.

**Calibration du noyau:**

L'approche proposée par Fearnhead et Taylor (2013) pour la calibration des paramètres $h_t = (\epsilon_t, L_t)$ du noyau $\mathcal{K}_t^h$ pour la méthode SMC repose sur l'hypothèse que les paramètres $h$ du noyau utilisé au temps $t-1$ permettront d'avoir de bonnes performances au temps $t$.

Dans la suite, on se place donc au temps $t-1$ de l'algorithme SMC.

Ici, l'article reprend cette approche en la modifiant et en l'adaptant.

<!-- **1ere Etape :** -->

* D'abord, on applique un premier HMC (pre-run) à chaque particule $x_i$, avec, pour chacune, un $(\epsilon_i, L_i)$ différent, tirés aléatoirement.
* On construit une nouvelle distribution $(\epsilon, L)$ basé sur les performances des $(\epsilon_i, L_i)_{i=1,...N}$. On utilise l'estimateur de Rao-Blackwellized comme métrique de performance.
* On supprime les résultats du HMC obtenus.

<!-- **2eme Etape :**  -->
* On applique HMC aux $N$ particules, avec les paramètres $(\epsilon, L)$, ce qui nous fait passer de $t-1$ à $t$.

Dans le meme esprit que Fearnhead et Taylor (2013), nous allons utiliser un estimateur qui nous permettra de paramétriser la sortie de l'algorithme. C'est à dire, d'assigner des poids $\omega_{t}^{i}$ au couple $(\epsilon_{t}^{i},L_{t}^{i})$.

L'estimateur de Rao-Blackwellized s'écrit :

$$
\tilde{\Lambda} (\tilde{x}_{t-1}^i, \hat{x}_t^i) = \frac{\lVert \tilde{x}_{t-1}^i - \hat{x}_t^i \rVert _M^2}{L} \times \min(1, \exp (\Delta E_t^i))
$$

où $\hat{x}_t^i$ désigne la particule proposée par le flot Hamiltonien $\varphi_{\epsilon, L} (\tilde{x}_{t-1}^i, p_t^i)$ avant l'étape de Metropolis Hasting et ${\lVert . \rVert _M}$ désigne la distance de Mahalanobis.

On note que le $\min(1, \exp (\Delta E_t^i))$ sera utilisé pour les poids et est le taux d'acceptation du MH selon $\Delta E_t^i$.

* Dans la première étape, on doit générer des $\epsilon$ et des $L$ uniformément. En particulier, on prend : $\epsilon \sim \mathcal{U}([0, \epsilon^\star])$ et $L\sim \mathcal{U}([0, L_{max}])$ pour des certains $\epsilon^\star$ et $L_{max}$ fixés.

* Le processus pour calculer $\epsilon^\star$ est le suivant : 
L'idée est que si on observe différents $\Delta E_t^i$ pour différentes valeurs de $\hat{\epsilon}_t^i$ et différents $(p_t^i, \tilde{x}_{t-1}^i)$, alors on peut les utiliser pour ajuster un modèle de regression de la forme $|\Delta E_t^i| = f(\hat{\epsilon}_t^i) + \xi_t^i$ où $f : \mathbb{R}^+ \mapsto \mathbb{R}^+$ et $\xi_t^i$ est une variable aléatoire centrée, de variance finie.

Ici, on considère un modèle polynomiale, paramétré par les coefficients $(\alpha_0, \alpha_1)$ : $f(\hat{\epsilon}_t^i) = \alpha_0 + \alpha_1 \hat{\epsilon}_t^{i2}$.

On cherche à minimiser l'erreur en norme L1: $\sum_{i=1}^N |\xi_t^i|$, ce qui revient à faire un regression médiane.

La raison de pourquoi on choisit d'estimer par la médiane est du à sa robustesse face aux outliers.

Enfin, on choisit $\epsilon^\star$ de façon à ce que $f(\epsilon^\star) = |\log 0.9|$. En effet, puisque $\epsilon^\star$ est la borne supérieure, on considère donc le pire cas, puisque plus $\epsilon$ est grand, plus les résultats sont mauvais. Ainsi en remplaçant cette valeurs dans $|\Delta E_t^i|$, on obtient un taux d'acceptation de 90% malgré le fait qu'on se place dans la borne supérieure.

$\textbf{Description de l'algorithme de Pre-tuning du noyau HMC}$

L'algorithme de Pre-tuning se déroule comme suit :

**Initialisation :**  On se situe à l'étape $t-1$ de l'algorithme SMC, et on dispose de particules $\{\tilde{x_{t-1}^i}\}_{i=1,...,N}$. On a également un flot de HMC $\varphi$, et un $\epsilon^\star_t$. 


**Pour chaque particule $i \in \{1,...,N\}$ :**

On tire $\hat{\epsilon}_t^i \sim \mathcal{U}([0, \epsilon^\star_t])$ et $\hat{L}_t^i \sim \mathcal{U}([0, L_{max}])$
On tire $q_t^i \sim \mathcal{N}(0, M_{t-1})$
On applique HMC pour obtenir $(\hat{q}_t^i, \hat{p}_t^i)$
On calcule $\Delta E_t^i$ et le critère de performance $\tilde{\Lambda}(\tilde{x}_{t-1}^i, \hat{x}_{t}^i)$

**A la fin :**
    On calcule $\epsilon_t^\star$ par quantile regression comme expliqué precedemment
    On calcule des poids $w_t^i$ proportionnels à $\tilde{\Lambda}(\tilde{x}_{t-1}^i, \hat{x}_{t}^i)$
    On resample les $\{\hat{\epsilon}_t^i, \hat{L}_t^i\}$ avec les poids $w_t^i$, notés  $\{{\epsilon}_t^i, {L}_t^i\}$

**Sortie :** $\{{\epsilon}_t^i, {L}_t^i\}$ et $\epsilon_t^\star$

## $\textbf{Experimentation}$ : 

In [5]:
N, n = 500, 5
XR, Y = sample_Y(n)
print(f"Vrai X : {XR}, Y : {Y}")
print(f"Esperance a posteriori réelle : {scipy.stats.beta(1 + Y.sum().cpu(), 1 + n - Y.sum().cpu()).mean()}")

class TunedHmcParameters:
    def __init__(self, eps_star, L, M, N):
        self.eps_star = eps_star
        self.epsilons = torch.rand((N, 1), device=device) * eps_star
        self.M = M
        self.L = L
        self.N = N

    def update_parameters(self, p0, q0, p, q, DeltaE):
        norm = torch.einsum('ij,jk,ik->i', q0, h.M, q0).unsqueeze(1)
        expDeltaE = torch.exp(DeltaE)
        lambd = norm * torch.where(expDeltaE <= 1, expDeltaE, torch.ones(DeltaE.shape, device=device))
        omega = lambd.squeeze().cpu().numpy()/lambd.cpu().numpy().sum()
        self.epsilons = torch.from_numpy(
            rng.choice(self.epsilons.cpu(), size=N, p=omega)
        ).to(device)

    def next_t(self):
        self.epsilons = torch.rand((self.N, 1), device=device) * self.eps_star

X, omegas = SMC(
    Y, sample_X, N, p, l, 
    K_HMC, TunedHmcParameters(0.1, 10, torch.tensor([[1.]], device=device), N),
    should_continue_markov, lambda t : lambda_get(10, t)
)
print(f"Estimateur de l'esperance a posteriori selon SMC : {X.mean()}")

Vrai X : tensor([0.4004]), Y : tensor([1., 0., 1., 0., 0.])
Esperance a posteriori réelle : 0.4285714328289032
Estimateur de l'esperance a posteriori selon SMC : 0.4100073277950287


## $\textit{Différences avec l'article}$ : 

Par souci de simplicité et d'implémentation, on a modifié certains points de l'algorithme et de la calibration présentés par les auteurs.
Les différences avec l'algorithme proposé par l'article sont les suivantes : 

- Quand on arrive dans la boucle pour le HMC, l'algorithme génère N $\epsilon_i \sim \mathcal{U}([0, \epsilon^\star])$ pour chaque particule. 

- A chaque itération du HMC dans l'algorithme implémenté, on calcule le critère de Rao-Blackwellized, les poids $w_i$ de chque $\epsilon_i$, et on resample les $\epsilon_i$ selon les performances. Les $\epsilon$ avec des petits poids disparaissent, ainsi on a progressivement une convergence vers une liste des $\epsilon_i$ où les poids sont les meilleurs, jusqu'à avoir 1 seul $\epsilon$.
- Le L est fixé et commun à toutes les particules dans l'algorithme implémenté.
- On n'implémente pas de regression médiane et $\epsilon^\star$ est fixé dès le début et n'évolue pas selon les itérations.
- Nous ne calibrons pas la matrice M

### $\textbf{IV - Conclusion}$

On a donc revu les méthodes SMC et HMC. On a introduit l'algorithme HMC dans SMC. Cet article propose des méthodes  de calibration de HMC dans SMC, que nous avons présentées et implémentées avec quelques simplifications. Notre calibration ne présente pas d'améliorations notables par rapport à la version dans calibration. Cela peut s'expliquer par les simplifications effectuées.

### $\textbf{Références}$ :

### $\textit{Alexander Buchholz, Nicolas Chopin, Pierre E. Jacob}$
   $\textit{MRC Biostatistics Unit, University of Cambridge}$
   $\textit{ENSAE-CREST}$
   $\textit{Department of Statistics, Harvard University}$

### $\textit{Jasra et al., 2011}$ 
### $\textit{Schäfer and Chopin, 2013}$
### $\textit{effective sample size - Kong et al., 1994}$
### $\textit{Agapiou et al., 2017}$

### $\textit{Fearnhead, P. and Taylor, B. M. (2013). *An adaptive sequential Monte Carlo sampler*.}$ 
   $\textit{Bayesian Analysis, 8(2):411–438}$


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0ce0e88e-d80d-4010-9886-0b33e6492711' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>